In [ ]:
import gensim, os, codecs, io


def iter_documents(top_directory):
    """Iterate over all documents, yielding a document (=list of utf8 tokens) at a time."""
    files = os.listdir(top_directory)
    print files
    for f in  files:
        document = []
        with open(top_directory + '/' + f, 'rb') as f:
            for line in f:
                try:
                    document.append(line)
                except:
                    pass
            document = ' '.join(document)
        yield gensim.utils.tokenize(document, lower=True, errors='ignore', deacc=True) # or whatever tokenization suits you

class MyCorpus(object):
    def __init__(self, top_dir):
        self.top_dir = top_dir
        self.dictionary = gensim.corpora.Dictionary(iter_documents(top_dir))
        self.dictionary.filter_extremes(no_below=1, keep_n=30000) # check API docs for pruning params

    def __iter__(self):
        for tokens in iter_documents(self.top_dir):
            yield self.dictionary.doc2bow(tokens)

corpus = MyCorpus('spanish_billion_words') # create a dictionary



In [3]:
gensim.corpora.MmCorpus.serialize('test.mm', corpus)
mm = gensim.corpora.MmCorpus('test.mm')
id2word = corpus.dictionary
print mm

['emoji_2016-09-30T09:30:13Z', 'emoji_2016-09-30T10:43:50Z', '0014', '0025', 'emoji_2016-09-30T10:30:25Z', '0017', '0005', '0009', '0004', '0028', '0008', '0010', '0002', '0019', 'emoji_2016-09-30T09:51:15Z', 'emoji_2016-09-30T09:14:54Z', '0007', 'emoji_2016-09-30T10:55:52Z', 'emoji_2016-09-30T10:04:07Z', '0029', '0021', '0015', '0030', '0024', '0013', '0003', '0006', '0023', '0026', 'emoji_2016-09-30T09:34:59Z', '0001', '0012', '0022', '0011', '0027', '0020', 'emoji_2016-09-30T10:16:27Z', '0016', '0018']
MmCorpus(39 documents, 30000 features, 171533 non-zero entries)


In [8]:
import numpy as np
#lsi = gensim.models.lsimodel.LsiModel(corpus=mm, id2word=id2word, num_topics=400, chunksize=20000, distributed=True)
lda = gensim.models.ldamulticore.LdaMulticore(corpus=mm,id2word=id2word, num_topics=50, passes=20, workers=13)

#lda.print_topics(20)

topics_matrix = lda.show_topics(formatted=False, num_words=20)
topics_matrix = np.array(topics_matrix)


ValueError: setting an array element with a sequence

In [3]:
#lda.print_topics(20)
topics_matrix = lda.show_topics(formatted=True, num_words=20)
print topics_matrix

NameError: name 'lda' is not defined

In [3]:
import re
class DirOfPlainTextCorpus(object):
    """Iterate over sentences of all plaintext files in a directory """
    SPLIT_SENTENCES = re.compile(u"[.!?:]\s+")  # split sentences on these characters

    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self):
        for fn in os.listdir(self.dirname):
            text = open(os.path.join(self.dirname, fn)).read()
            for sentence in self.SPLIT_SENTENCES.split(text):
                yield gensim.utils.simple_preprocess(sentence, deacc=True)

model = gensim.models.Word2Vec(DirOfPlainTextCorpus('emoji-tracker-dataset/data'), size=200, min_count=5, workers=10)

In [4]:
print model

Word2Vec(vocab=13637, size=200, alpha=0.025)


In [25]:
from sklearn import cluster
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.syn0
num_clusters = word_vectors.shape[0] / 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = cluster.KMeans( n_clusters = num_clusters, n_jobs=-2 )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print "Time taken for K Means clustering: ", elapsed, "seconds."

Time taken for K Means clustering:  276.917459965 seconds.


In [14]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.index2word, idx ))

In [15]:
# For the first 10 clusters
for cluster in xrange(0,10):
    #
    # Print the cluster number  
    print "\nCluster %d" % cluster
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( word_centroid_map.values()[i] == cluster ):
            words.append(word_centroid_map.keys()[i])
    print words


Cluster 0
[u'quitarte', u'serrr', u'apuntar', u'nuez', u'nazca', u'airbag', u'volverlo', u'impedir', u'colaborar', u'mery', u'alimentar', u'atraer', u'royal', u'abandonar', u'creerlo', u'pillar', u'groserias', u'picar', u'desnudarte', u'aportar', u'transmitir', u'pretender', u'estallar', u'obedecer', u'aplicar', u'llamarme', u'separar', u'previas']

Cluster 1
[u'sentimos', u'atrae', u'haha']

Cluster 2
[u'cineasta', u'tim']

Cluster 3
[u'tienen']

Cluster 4
[u'loco']

Cluster 5
[u'estudiar']

Cluster 6
[u'quieras']

Cluster 7
[u'dura']

Cluster 8
[u'basta']

Cluster 9
[u'haran', u'permiten', u'ayudan', u'goals', u'aparecen', u'opinas', u'suelen', u'falsas']


In [16]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids


In [18]:
import numpy as np
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

NameError: name 'train' is not defined

In [ ]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print "Fitting a random forest to labeled training data..."
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )